In [1]:
!nvidia-smi

Sat Aug 12 06:32:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.191.01   Driver Version: 450.191.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   28C    P0    58W / 300W |  10270MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from datasets import Dataset, DatasetDict

wow = DatasetDict.load_from_disk("wow")

In [4]:
import yake

language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

custom_kw_extractor_1 = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)


In [5]:
import spacy
def masker(inp, mask, ref_in, ref_out):
    op = inp
    msk = []
    j = 0
    for i, w in enumerate(mask):
        if w.lower() in ref_out.lower() and w.lower() not in ref_in.lower():
            op = op.replace(w, f"<extra_id_{j}>")
            j+=1
            msk.append(w)


    return op, " ".join(msk)

en = spacy.load('en_core_web_sm')
def key_exrt(k_lst, strng):

    keywords = []
    k2 = []
    if len(k_lst) == 0:
        keywords += [token for token in en(strng) if
              not token.is_punct
              and not token.is_currency
              and not token.is_digit
              and not token.is_oov
              and not token.is_space
              and not token.is_stop
              and not token.like_num
              and not token.pos_ in []]
    else:
        keywords += [key for key, _ in k_lst]
        k2 += [key for key, _ in k_lst]

    return keywords, ("; ".join(keywords) + ";"), k2

In [6]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-large")
question = "question:"
title = "title:"
context = "context:"
eou = "<eou>"
tokenizer.add_tokens([question, title, context, eou], special_tokens=True)

/raid/kavin-intern-maunendra/anaconda3/envs/wow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to re

4

In [7]:
import torch
def prep1(df):
    out = {"pass_label": [], "attention_mask": [], "input_ids": []}
    for i, _ in enumerate(df["gold_pass"]):
        # print(type(k))
        if df["gold_pass"][i] == float("-inf"):
            continue
#         keys = custom_kw_extractor_1.extract_keywords(df["response"][i])
#         key, strng, js_ks = key_exrt(keys, df["response"][i])
#         pas = df["all_pass"][i][int(df["gold_pass"][i])] if df["gold_pass"][i] != float("-inf") else ""
#         msk_res, gpe_out = masker(df["response"][i], js_ks, df["context_eou"][i].replace(" <eou>", ""), pas)
        input_q = df["last_ut"][i] 
        input_ids = []
        attention_mask = []
#         out["pass_label"].append(int(df["gold_pass"][i]))
        for j, p in enumerate(df["all_pass"][i]):

            if j == int(df["gold_pass"][i]):
                # pass_label.append(1)
                p = (" ".join(df["all_sen"][i]))

            p = p.replace("no_passages_used", "")
            t = df["all_topic"][i][j]
            inp = tokenizer(f"question: {input_q} title: {t} passage: {p}", max_length=512, truncation=True, padding=False)
            out["input_ids"].append(inp["input_ids"])
            out["attention_mask"].append(inp["attention_mask"])
        
    #         le = 0
    #         for k in input_ids:
    #             if len(k) > le:
    #                 le = len(k)

    #         input_ids = [m + [tokenizer.pad_token_id]*(le-len(m)) for m in input_ids]
    #         attention_mask = [m + [tokenizer.pad_token_id]*(le-len(m)) for m in attention_mask]
            
            out["pass_label"].append(int(df["gold_pass"][i]))
#             out["labels"].append(tokenizer(gpe_out)["input_ids"])
#         out["input_ids"].append(torch.tensor(input_ids))
#         out["attention_mask"].append(torch.tensor(attention_mask))
        # out["masked_response"].append(input_q)
        # out["resp"].append(df["response"][i])

    return out

In [8]:
tokenized_wow = wow.map(prep1, batched=True, remove_columns=wow["train"].column_names)

Map:   0%|          | 0/74092 [00:00<?, ? examples/s]

Map:   0%|          | 0/3939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3865 [00:00<?, ? examples/s]

In [ ]:
# tokenized_wow.rename_column()

In [10]:
tokenized_wow.save_to_disk("wow_rank_ut_base")

Saving the dataset (0/2 shards):   0%|          | 0/491316 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/26313 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25732 [00:00<?, ? examples/s]

In [9]:
tokenized_wow.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "pass_label"])

In [10]:
tokenized_wow

DatasetDict({
    train: Dataset({
        features: ['pass_label', 'labels', 'attention_mask', 'input_ids'],
        num_rows: 491316
    })
    valid: Dataset({
        features: ['pass_label', 'labels', 'attention_mask', 'input_ids'],
        num_rows: 26313
    })
    test: Dataset({
        features: ['pass_label', 'labels', 'attention_mask', 'input_ids'],
        num_rows: 25732
    })
})

In [11]:
tokenized_wow["train"]["labels"][:7]

[tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1]),
 tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1]),
 tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1]),
 tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1]),
 tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1]),
 tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1]),
 tensor([  569,  1827, 22466, 17201,    18,    89,    23,     1])]

Model

In [12]:
from transformers import T5ForConditionalGeneration, T5Config
import torch
from torch import nn

class RankT5GPE(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
        config.rank_score_index = None
        config.n_pass = None
        super().__init__(config)
        self.rank_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.rank_id = config.rank_score_index
        self.n_pass = config.n_pass

    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, pass_label=None, **kwargs):

        batch_size_n, seq_len = input_ids.size()
        batch_size = batch_size_n/self.n_pass

        # input_ids = input_ids.view(batch_size*n_pass, -1)
        # attention_mask = attention_mask.view(batch_size*n_pass, -1)
        
        
        
        if labels != None and decoder_input_ids == None:
#             batch_size, decoder_seq_len = labels.size()
#             labels = labels.view(batch_size, 1, decoder_seq_len).contiguous()
#             labels = labels.expand(batch_size, n_pass, decoder_seq_len).contiguous()

#             labels = labels.view(batch_size*n_pass, -1)
            decoder_input_ids = self._shift_right(labels)
#             print(decoder_input_ids.size())


        out = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
        rank_score = self.rank_head(out.decoder_hidden_states[-1][:, 0, :])
        out.rank_score = rank_score[:, self.rank_id].view(-1, self.n_pass)


        if labels != None:
            logits = out.logits
            batch_size_n, sequence_length, vocab_size = logits.size()

            logits_flat = logits.view(batch_size_n * sequence_length, vocab_size)
            labels_flat = labels.view(-1)
            mask = (labels_flat != -100)

            selected_logits = logits_flat[torch.arange(batch_size_n * sequence_length)[mask], labels_flat[mask]]
            output_logits = torch.full((batch_size_n * sequence_length,), 0, dtype=logits.dtype, device=logits.device)
            output_logits[mask] = selected_logits

            output_logits = output_logits.view(batch_size_n, -1).sum(-1)
            out.gpe_score = output_logits.view(int(batch_size_n/self.n_pass), self.n_pass)

        else:
            out.gpe_score = None


        if pass_label != None:
            pass_label = pass_label[::self.n_pass]
            rank_score = out.rank_score
            gen_score = out.gpe_score

            loss_fct1 = nn.CrossEntropyLoss()
            loss_fct2 = nn.CrossEntropyLoss()

            rank_loss = loss_fct1(rank_score, pass_label.view(-1))
            gen_loss = loss_fct2(gen_score, pass_label.view(-1))

            loss = rank_loss + gen_loss
            out.loss = loss

        return out

In [13]:
pass_label = torch.randint(0, 5, (2,))
pass_label.view(2, 1)

tensor([[0],
        [4]])

In [14]:
logits = torch.randint(0, 32127, (10, 10))
lab = torch.randint(0, 32127, (2, 6))
lab = lab.view(2, 1, 6).contiguous()
lab = lab.expand(2, 5, 6).contiguous()
lab = lab.view(10, 6)
pass_label = torch.randint(0, 5, (2,))
pass_label = pass_label.view(2, 1)
pass_label = pass_label.expand(2, 5).contiguous()
pass_label = pass_label.view(10)

In [15]:
pass_label

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [16]:
from transformers import T5Tokenizer

mod_ckp = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(mod_ckp)
config = T5Config.from_pretrained(mod_ckp)
model = RankT5GPE(config).from_pretrained(mod_ckp)
model.config.output_hidden_states = True
model.config.rank_score_index = tokenizer.convert_tokens_to_ids("<extra_id_80>")
model.config.n_pass = 7

# x = m(input_ids=logits, attention_mask=torch.ones_like(logits), pass_label=pass_label, labels=lab)

Some weights of RankT5GPE were not initialized from the model checkpoint at t5-small and are newly initialized: ['rank_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

batch_size = 8*model.config.n_pass 
model_dir = f"wow_rank_{mod_ckp}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    save_strategy="steps",
    save_steps=5000,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="tensorboard",
    gradient_accumulation_steps=4
)

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_wow["train"],
    eval_dataset=tokenized_wow["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

RuntimeError: TensorBoardCallback requires tensorboard to be installed. Either update your PyTorch version or install tensorboardX.

In [44]:
tra

'▁Cro'

In [50]:
tokenizer.decode(tokenized_wow["input_ids"][500])

"question: Oh that's awesome. I have a corgi who is a year old. She's very well behaved and easily trainable. <eou> That is a dog<extra_id_2> to mines I have a<extra_id_1> corgi it is a much smaller dog though<extra_id_3> from<extra_id_0>. title: Cardigan Welsh Corgi passage: The Cardigan Welsh Corgi is one of two separate dog breeds known as Welsh corgis that originated in Wales, predating the other breed, the Pembroke Welsh Corgi. It is one of the oldest breeds of the British Isles. Cardigan Welsh Corgis are known to be an extremely loyal dog breed. They are also versatile and can live in a variety of settings. However, they benefit from regular physical and mental stimulation. Pembrokes and Cardigans first appeared together in dog shows in 1925 when they were shown under the rules of The Kennel Club in Britain. The Corgi Club was founded in December, 1925 in Carmarthen in South Wales.</s>"

In [51]:
tokenizer.convert_tokens_to_ids("<eou>")

32103

In [69]:
tokenizer.convert_ids_to_tokens(31999+1)

'<extra_id_99>'